# Pre-requisite Configuration

Following pre-requisites are performed before the extraction of data from relevant systems.

1. **Set Configuration:**
    * Set a `CONFIG_ID` for execution.
    * Code picks & loads configuration file (yaml) from `config` folder based on the `config_id` maintained in the file(s). 
2. **Create directories:**
    * Pick the extraction directory maintained in configuration for Indicators
    * Extraction data such as CSV / parquet files are staged in this directory.
    * This data is stored locally.
    * A `reports` directory is also created within, where the error reports are stored.
3. **Create database tables:**
    * The script creates necessary tables in the database specified in the configuration.
    * Tables are only created if they **do not** already exist in the database.

In [36]:
# Pre-requisite Configuration
# ------------------------------------------------------------------------------ #
# For testing
# ------------------------------------------------------------------------------ #

%load_ext autoreload
%autoreload 2


from modules.util.config import get_config_by_id
from modules.util.database import SQLAlchemyClient
from modules.util.helpers import Logger
import os 

# ------------------------------------------------------------------------------ #
# Configuration
# ------------------------------------------------------------------------------ #

CONFIG_ID = 'dev'

config_extract = get_config_by_id(CONFIG_ID).get("extract")
config_indicators = config_extract['indicator']
EXTRACTION_DIR = config_indicators["directory"]
REPORTS_DIR = f"{EXTRACTION_DIR}/reports"

# ------------------------------------------------------------------------------ #
# Directories
# ------------------------------------------------------------------------------ #
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)

log = Logger.get_logger(CONFIG_ID)
log.info("** EXTRACT - INDICATORS **")
Logger.blank_line(log)
log.info(f"Configuration ID: {CONFIG_ID}")
log.info(f"Extraction directory: {EXTRACTION_DIR}")
log.info(f"Reports directory: {REPORTS_DIR}")

# ------------------------------------------------------------------------------ #
# Database
# ------------------------------------------------------------------------------ #

db = SQLAlchemyClient(CONFIG_ID)
db.table_create_all()

2025-01-25 18:37:33,280  [INFO]: ** EXTRACT - INDICATORS **

2025-01-25 18:37:33,282  [INFO]: Configuration ID: dev
2025-01-25 18:37:33,282  [INFO]: Extraction directory: ../migration-data/indicators/extract
2025-01-25 18:37:33,282  [INFO]: Reports directory: ../migration-data/indicators/extract/reports
2025-01-25 18:37:33,301  [INFO]: [DB] Tenant ID: dev
2025-01-25 18:37:33,302  [INFO]: [DB] Database Connection: sqlite:///../migration-data/dev.db
2025-01-25 18:37:33,302  [DEBU]: [DB] SQLAlchemy Echo: False
2025-01-25 18:37:33,302  [INFO]: [DB] Drop Reload: True
2025-01-25 18:37:33,346  [DEBU]: [DB] Table Created: T_PAI_EXTERNALDATA
2025-01-25 18:37:33,346  [DEBU]: [DB] Table Created: T_PAI_EXTERNALDATA_FLOC
2025-01-25 18:37:33,347  [DEBU]: [DB] Table Created: T_EIOT_MAPPING
2025-01-25 18:37:33,347  [DEBU]: [DB] Table Created: T_EIOT_MAPPING_INDICATORS
2025-01-25 18:37:33,347  [DEBU]: [DB] Table Created: T_EIOT_UPLOAD_STATUS
2025-01-25 18:37:33,348  [DEBU]: [DB] Table Created: T_APM_IN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1 - Extract: Technical Objects

* Technical Objects consists of Equipments & Functional Locations. For migration we consider only the technical objects which have a valid `templateId` & which are either in the `published` or `in revision` state.
* Hence while extracting technical objects (EQUs / FLOCs) from the source system, we apply the following filters :
    * `templateId is not blank`
    * `status is not 1` (ignore the records with unpublished status)

In [37]:
# Technical Objets: Extraction [Equipments / Functional Locations]
# ------------------------------------------------------------------------------ #
# Standard Imports
# ------------------------------------------------------------------------------ #
import pandas as pd

# ------------------------------------------------------------------------------ #
# Custom Imports
# ------------------------------------------------------------------------------ #
from modules.acf.equ_api import ApiEquipment
from modules.acf.floc_api import ApiFloc

# ------------------------------------------------------------------------------ #
# Extraction
# ------------------------------------------------------------------------------ #

file_equ = rf'{EXTRACTION_DIR}/1_Equipments.parquet'
file_floc = rf'{EXTRACTION_DIR}/1_Functional_Locations.parquet'
filter = "not(templateId eq 'null') and not(status eq '1')"
batch_size = 1000

api_equ = ApiEquipment(CONFIG_ID)
ApiFloc = ApiFloc(CONFIG_ID)

Logger.blank_line(log)
log.info("Extract: Technical Objects [Equipments / Functional Locations]")
Logger.blank_line(log)

# Extract & generate data file for Equipments
results = api_equ.get_equipments(
    filter=filter,
    batch_size=batch_size
)

df_equ = pd.DataFrame(results)
df_equ.rename(columns={'class': 'class_'}, inplace=True) #due to constraint in python & SQL

# inclusion list for equipments
equ_include = config_extract.get('filters').get('equipments_include')
if equ_include:
    log.warning(f"[FILTER] Equipments - include: {equ_include}")
    df_equ = df_equ[df_equ['equipmentId'].isin(equ_include)]

# exclusion list for equipments
equ_exclude = config_extract.get('filters').get('equipments_exclude')
if equ_exclude:
    log.warning(f"[FILTER] Equipments - exclude: {equ_exclude}")
    df_equ = df_equ[~df_equ['equipmentId'].isin(equ_exclude)]

if equ_include or equ_exclude:
    log.info(f"Total Equipments: {len(df_equ)}")

df_equ.to_parquet(file_equ)
log.info(f"{file_equ} generated.")

# Extract & generate data file for Functional Locations
results = ApiFloc.get_flocs(
    filter=filter,
    batch_size=batch_size
)

df_floc = pd.DataFrame(results)
df_floc.rename(columns={'class': 'class_'}, inplace=True)
df_floc.to_parquet(file_floc)
log.info(f"{file_floc} generated.")


2025-01-25 18:37:33,461  [INFO]: Extract: Technical Objects [Equipments / Functional Locations]

2025-01-25 18:37:34,906  [INFO]: [GET] Equipments: 1688
2025-01-25 18:37:34,969  [INFO]: ../migration-data/indicators/extract/1_Equipments.parquet generated.
2025-01-25 18:37:35,348  [INFO]: [GET] Functional Locations: 33
2025-01-25 18:37:35,362  [INFO]: ../migration-data/indicators/extract/1_Functional_Locations.parquet generated.


The data fetched above with the necessary filters are then staged in the corresponding tables in the database.

| Data | Table Name |
| ---- | ---------- |
| Equipments | `T_PAI_EQU_HEADER`
| Functional Locations | `T_PAI_FLOC_HEADER`


In [38]:
# Technical Objects: Stage Data in Tables
# ------------------------------------------------------------------------------ #

from modules.util.database import SQLAlchemyClient, EquipmentHeader, FlocHeader

if db.drop_reload:
    db.truncate(EquipmentHeader)
    db.truncate(FlocHeader)

equipments = SQLAlchemyClient.dataframe_to_object(df_equ, EquipmentHeader)
flocs = SQLAlchemyClient.dataframe_to_object(df_floc, FlocHeader)

if equipments:
    db.insert_batches(data=equipments)

if flocs:
    db.insert_batches(data=flocs)

2025-01-25 18:37:35,375  [WARN]: [DB] TRUNCATE T_PAI_EQU_HEADER: 0
2025-01-25 18:37:35,376  [WARN]: [DB] TRUNCATE T_PAI_FLOC_HEADER: 0
2025-01-25 18:37:35,498  [INFO]: [DB] INSERT T_PAI_EQU_HEADER: 1688
2025-01-25 18:37:35,555  [INFO]: [DB] INSERT T_PAI_FLOC_HEADER: 33


# 2 - Extract: External Data

* During migration, we consider only the Technical Objects which have an **external ID** assigned. 
* We call the `ExternalID` API to extract all the external IDs maintained in the system for Equipments & Functional Locations. 
* This data will then be used as a *check table* for migrating only the records which have a valid External ID maintained. 
* During the external ID extraction, we apply the following filters.
    * `systemType is 'ERP'`
    * `systemName is <sap-system-id>_<sap-client-number>`. The SAP system ID and client number are provided in the configuration
    * `externalId does NOT contain LOCAL_`. *LOCAL_* are technical objects that are local to the source systems
    * `objectType is 'EQU' or 'FLOC'`. EQU represents equipments & FLOC represents Functional Locations.

In [39]:
# External Data: Extraction [Equipments / Functional Locations]
# ------------------------------------------------------------------------------ #

# Standard Imports
import pandas as pd

# Custom Imports
from modules.acf.external_id_api import ApiExternalId


file_equ = rf'{EXTRACTION_DIR}/2_External_Data_EQU.parquet'
file_floc = rf'{EXTRACTION_DIR}/2_External_Data_FLOC.parquet'

api = ApiExternalId(CONFIG_ID)
filter = f"systemType eq 'SAP ERP' and systemName eq '{api.erp_ssid}' and not substringof('LOCAL_', externalId)"

Logger.blank_line(log)
log.info("Extract: External Data [Equipments / Functional Locations]")
Logger.blank_line(log)

def extract_externalData(object_type, filter_base):
    filter_str = f"{filter_base} and objectType eq '{object_type}'"
    results = api.get_external_data(
        filter_str=filter_str,
        batch_size=20000
    )
    return results

results_equ = extract_externalData('EQU', filter)
df_equ_external = pd.json_normalize(results_equ)
df_equ_external.to_parquet(file_equ)
log.info(f"{file_equ} generated.")


results_floc = extract_externalData('FLOC', filter)
df_floc_external = pd.json_normalize(results_floc)
df_floc_external.to_parquet(file_floc)
log.info(f"{file_floc} generated.")


2025-01-25 18:37:35,594  [INFO]: Extract: External Data [Equipments / Functional Locations]

2025-01-25 18:37:35,930  [INFO]: [GET] External Data systemType eq 'SAP ERP' and systemName eq 'QM7_910' and not substringof('LOCAL_', externalId) and objectType eq 'EQU': 3
2025-01-25 18:37:35,940  [INFO]: ../migration-data/indicators/extract/2_External_Data_EQU.parquet generated.
2025-01-25 18:37:36,058  [INFO]: [GET] External Data systemType eq 'SAP ERP' and systemName eq 'QM7_910' and not substringof('LOCAL_', externalId) and objectType eq 'FLOC': 1
2025-01-25 18:37:36,067  [INFO]: ../migration-data/indicators/extract/2_External_Data_FLOC.parquet generated.


The data fetched above with the necessary filters are then staged in the corresponding tables in the database.

| Data | Table Name |
| ---- | ---------- |
| External Data for Equipments | `T_PAI_EXTERNALDATA`
| External Data for Functional Locations | `T_PAI_EXTERNALDATA_FLOC`


In [40]:
# External Data: Stage Data in Tables
# ------------------------------------------------------------------------------ #

from modules.util.database import ExternalData_EQU, ExternalData_FLOC

if db.drop_reload:
    db.truncate(ExternalData_EQU)
    db.truncate(ExternalData_FLOC)

external_equipments = SQLAlchemyClient.dataframe_to_object(df_equ_external, ExternalData_EQU)
if external_equipments:
    db.insert_batches(data=external_equipments)

external_flocs = SQLAlchemyClient.dataframe_to_object(df_floc_external, ExternalData_FLOC)
if external_flocs:       
    db.insert_batches(data=external_flocs)

2025-01-25 18:37:36,080  [WARN]: [DB] TRUNCATE T_PAI_EXTERNALDATA: 0
2025-01-25 18:37:36,081  [WARN]: [DB] TRUNCATE T_PAI_EXTERNALDATA_FLOC: 0
2025-01-25 18:37:36,082  [INFO]: [DB] INSERT T_PAI_EXTERNALDATA: 3
2025-01-25 18:37:36,084  [INFO]: [DB] INSERT T_PAI_EXTERNALDATA_FLOC: 1


- Once the relevant data is staged, we build views that establish a relationship between the extracted technical objects with the external data that was extracted from the system.
- These views help us to filter the data, viz. perform further steps only for the technical objects that have an external ID maintained.
- The views join between the tables to establish the relationship.
- They have a `valid` column present, which represents if the records are valid for processing or not.
    - `valid` = `X` if the technical object has a external ID maintained.
    - `valid` = `NULL` if the technical object does NOT have an external ID maintained.

# Model Extraction - Generic Method

A generic implementation to extract model information linked to technical objects. This method will be used in the upcoming steps.

In [41]:
# Models: Extraction (Generic Method)
# ------------------------------------------------------------------------------ #

# Standard Imports
from concurrent.futures import ThreadPoolExecutor, as_completed

# Custom Imports
from modules.acf.model_api import ApiModel
from modules.util.api import APIException

def extract_models(results:list, modelColumn:str = 'modelId')->list:

    """
    Extracts unique model information from the given models and fetches their details using an API.
    Args:
        results (list): A list of dictionaries containing model IDs.
        modelColumn (str, optional): The column name in the results that contains the model IDs. Defaults to 'modelId'.
    Returns:
        list: A tuple containing two lists:
            - response: A list of model details fetched from the API.
            - error: A list of errors encountered during the API calls, each error is a dictionary with keys 'id', 'status_code', 'response', and 'endpoint'.
    Raises:
        APIException: If an error occurs during the API call.
    Example:
        >>> results = [{'modelId': '123'}, {'modelId': '456'}, {'modelId': '123'}]
        >>> response, error = extract_models(results)
        >>> print(response)
        >>> print(error)
    """

    df_models = pd.DataFrame(results)
    df_models.drop_duplicates(subset=[modelColumn], inplace=True)

    log.info(f'Unique models linked to TOs: {len(df_models)}')

    api = ApiModel(CONFIG_ID)

    response = []
    error = []

    log.info('[GET] Models based on Model ID')

    def call_api(id):
        return(api.get_model_model_id(id))
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        future_id = {executor.submit(call_api, id): id for id in df_models[modelColumn]}
        for future in as_completed(future_id):
            id = future_id[future]
            try:
                res = future.result()
                if res:
                    response.append(res)
            except APIException as api_e:
                log.error(f"API Exception: {id} - {api_e.status_code} - {api_e.response} - {api_e.endpoint}")
                err = {
                    'id': id,
                    'status_code': api_e.status_code,
                    'response':api_e.response,
                    'endpoint':api_e.endpoint
                }
                error.append(err)
    log.info(f'Model details fetched: {len(response)}')
    if error:
        log.error(f'Models (with errors): {len(error)}')
    return response, error


# 3 - Extract Model Details

In this section, we extract model details for the technical objects (Equipments and Functional Locations) that have been identified as valid for processing. The extraction process involves the following steps:

1. **Fetch Model IDs:**
    - We fetch unique model IDs linked to the technical objects from the database views `V_PAI_EquipmentExternalData` and `V_PAI_FlocExternalData`.

2. **Extract Model Details:**
    - Using the fetched model IDs, we call the API to get detailed information about each model.
    - The extracted model details are normalized and stored in parquet files for further processing.

3. **Handle Errors:**
    - Any errors encountered during the API calls are logged and stored in CSV files for review.

4. **Stage Data in Tables:**
    - The extracted model details are then staged in the corresponding tables in the database.

The following table summarizes the data extraction and staging process:

| Data | Table Name |
| ---- | ---------- |
| Model Header for Equipments | `T_PAI_EQU_MODEL_TEMPLATES` |
| Model Header for Functional Locations | `T_PAI_FLOC_MODEL_TEMPLATES` |

The extracted data is crucial for the subsequent steps, where we will extract and process templates, indicator groups, and indicators linked to these models.

In [42]:
# Models: Extraction
# ------------------------------------------------------------------------------ #

# Standard Imports
import pandas as pd

# Custom Imports
from modules.util.database import V_PAI_EquipmentExternalData, V_PAI_FlocExternalData,EquModelTemplates, FlocModelTemplates, SQLAlchemyClient
from modules.util.helpers import convert_dataframe, explode_normalize

file_equ_model = rf'{EXTRACTION_DIR}/3_Model_Header_EQU.parquet'
file_equ_err = rf'{REPORTS_DIR}/3_Model_EQU_errors.csv'

file_floc_model = rf'{EXTRACTION_DIR}/3_Model_Header_FLOC.parquet'
file_floc_err = rf'{REPORTS_DIR}/3_Model_FLOC_errors.csv'

def process_data(model, table, success_file, error_file):
    results = db.select(
        model=model,
        fields=["modelId"],
        distinct=True,
        where=[model.valid == "X"],
        orderby=['modelId']
    )

    if len(results) > 0:
        response,error = extract_models(results)
        df_model_normal = pd.json_normalize(response, sep="_")

        df_model_explode = explode_normalize(
        data=df_model_normal,
        id=[col for col in df_model_normal.columns if col != 'templates'],
        id_explode='templates')

        df_model_explode = convert_dataframe(df_model_explode)
        df_model_explode.rename(columns={'class': '_class'}, inplace=True)
        
        df_model_explode.to_parquet(success_file)
        log.info(f"{success_file} generated.")
        
        if db.drop_reload is True:
            db.truncate(table)
        
        models = SQLAlchemyClient.dataframe_to_object(df_model_explode, table)
        if models:
            db.insert_batches(data=models)

        if error:
            df_error = pd.DataFrame(error)
            df_error.to_csv(error_file, index=False)
            log.error(f"{error_file} generated.")

Logger.blank_line(log)
log.info("Extract: Model Header for Equipments")
Logger.blank_line(log)
process_data(V_PAI_EquipmentExternalData, EquModelTemplates, file_equ_model, file_equ_err)

Logger.blank_line(log)
log.info("Extract: Model Header for Functional Locations")
Logger.blank_line(log)
process_data(V_PAI_FlocExternalData, FlocModelTemplates, file_floc_model, file_floc_err)


2025-01-25 18:37:36,116  [INFO]: Extract: Model Header for Equipments

2025-01-25 18:37:36,118  [DEBU]: SELECT DISTINCT "V_PAI_EQU_EXTERNAL_DATA"."modelId" 
FROM "V_PAI_EQU_EXTERNAL_DATA" 
WHERE "V_PAI_EQU_EXTERNAL_DATA".tenantid = :tenantid_1 AND "V_PAI_EQU_EXTERNAL_DATA".valid = :valid_1 ORDER BY "V_PAI_EQU_EXTERNAL_DATA"."modelId"
2025-01-25 18:37:36,131  [INFO]: [DB] SELECT V_PAI_EQU_EXTERNAL_DATA: 3
2025-01-25 18:37:36,138  [INFO]: Unique models linked to TOs: 3
2025-01-25 18:37:36,166  [INFO]: [GET] Models based on Model ID
2025-01-25 18:37:36,588  [INFO]: Model details fetched: 3
2025-01-25 18:37:36,607  [INFO]: ../migration-data/indicators/extract/3_Model_Header_EQU.parquet generated.
2025-01-25 18:37:36,608  [WARN]: [DB] TRUNCATE T_PAI_EQU_MODEL_TEMPLATES: 0
2025-01-25 18:37:36,609  [INFO]: [DB] INSERT T_PAI_EQU_MODEL_TEMPLATES: 4

2025-01-25 18:37:36,612  [INFO]: Extract: Model Header for Functional Locations

2025-01-25 18:37:36,613  [DEBU]: SELECT DISTINCT "V_PAI_FLOC_EXTE

# Template Extraction - Generic Method

A generic implementation to extract template information linked to technical objects. This method will be used in the upcoming steps.

In [43]:
# Templates: Extraction (Generic Method)
# ------------------------------------------------------------------------------ #

# Standard Imports
# from concurrent.futures import ThreadPoolExecutor, as_completed

# Custom Imports
from modules.acf.template_api import ApiTemplate
from modules.util.api import APIException

def extract_templates(results:list, templateColumn:str='templateId') -> list:
    """
    Extracts unique templates from the provided results and fetches detailed data for each template using an API.
    Args:
        results (list): A list of dictionaries containing template data (it should contain an entry with 'templateId' field).
    Returns:
        tuple: A tuple containing two lists:
            - response (list): A list of dictionaries with detailed template data.
            - error (list): A list of dictionaries with error information for templates that failed to fetch.
    Raises:
        APIException: If there is an issue with the API call.
        Exception: For any other exceptions that occur during the API call.
    """
    
    df_template = pd.DataFrame(results)
    df_template.drop_duplicates(subset=[templateColumn], inplace=True)

    log.info(f'Unique Templates linked to TOs: {len(df_template)}')

    api = ApiTemplate(CONFIG_ID)

    response = []
    error = []

    log.info('[GET] Templates based on Template ID')

    def call_api(id):
        return(api.get_template_template_id(id))

    with ThreadPoolExecutor(max_workers=20) as executor:
        future_id = {executor.submit(call_api, id): id for id in df_template[templateColumn]}
        for future in as_completed(future_id):
            id = future_id[future]
            try:
                res = future.result()
                if res:
                    #since there can be more than one responses for a templateId, we need to loop through all responses
                    for data in res:
                        template_with_id = {templateColumn: id, **data}
                        response.append(template_with_id)
            except APIException as api_e:
                log.error(f"API Exception: {id} - {api_e.status_code} - {api_e.response} - {api_e.endpoint}")
                err = {
                    'id': id,
                    'status_code': api_e.status_code,
                    'response':api_e.response,
                    'endpoint':api_e.endpoint
                }
                error.append(err)
    log.info(f'Templates fetched: {len(response)}')
    if error:
        log.error(f'Templates(with errors): {len(error)}')
    return response, error
    

# 3 - EQU Templates (Indicator Groups & Indicators)

To extract template information for Equipments, we follow these steps:

1. **Fetch Template Data:**
    - We fetch template data from the view `V_PAI_EquipmentExternalData` for Equipments that are valid for processing.

2. **Extract Templates:**
    - Using the fetched template data, we call the API to get detailed information about each template.
    - The extracted template details are normalized and stored in parquet files for further processing.

3. **Handle Errors:**
    - Any errors encountered during the API calls are logged and stored in CSV files for review.

4. **Stage Data in Tables:**
    - The extracted template details are then staged in the corresponding tables in the database.

5. **Extract Indicator Groups and Indicators:**
    - For each template, we extract the associated indicator groups and indicators.
    - The extracted data is normalized and stored in parquet files.
    - The data is then staged in the corresponding tables in the database.

The following table summarizes the data extraction and staging process:

| Data | Table Name |
| ---- | ---------- |
| Template Header for Equipments | `T_PAI_EQU_TEMPLATE_HEADER` |
| Indicator Groups for Equipments | `T_PAI_EQU_INDICATOR_GROUPS` |
| Indicators for Equipments | `T_PAI_EQU_INDICATORS` |

The extracted data is crucial for the subsequent steps, where we will process the indicators linked to these templates.

In [44]:
# [EQU] Templates: Extraction
# ------------------------------------------------------------------------------ #

# Standard Imports
import pandas as pd

# Custom Imports
from modules.util.database import SQLAlchemyClient, EquTemplateHeader, V_PAI_EquipmentExternalData
# from modules.util.helpers import explode_normalize, convert_dataframe

file_tgt = rf'{EXTRACTION_DIR}/3_Template_EQU.parquet'
file_err = rf'{REPORTS_DIR}/3_Template_EQU_Errors.csv'
df_template_normal = pd.DataFrame()

Logger.blank_line(log)
log.info("Extract: Templates - Equipments [Indicator Groups > Indicators]")
Logger.blank_line(log)

# fetch template data from the view for external data - equipments
results = db.select(
    model=V_PAI_EquipmentExternalData,
    fields=['id', 'externalId', 'templateId', 'valid'],
    distinct=True,
    where=[V_PAI_EquipmentExternalData.valid == "X"],
    orderby=['externalId']
)

if len(results) > 0:

    # extract templates for the identified records
    response,error = extract_templates(results)

    df_template_normal = pd.json_normalize(response, sep='_')
    df_template_normal = convert_dataframe(df_template_normal)
    df_template_normal.to_parquet(file_tgt, index=False)
    log.info(f"{file_tgt} generated")

    if error is not None and error != []:
        df_err = pd.json_normalize(error)
        df_err.to_csv(file_err, index=False)
        log.error(f"{file_err} generated")

    if db.drop_reload:
        db.truncate(EquTemplateHeader)  # truncate all existing data (remove if needed)
    templates = SQLAlchemyClient.dataframe_to_object(df_template_normal, EquTemplateHeader)
    if templates:
        db.insert_batches(data=templates)  # insert the new templates
    else:
        log.warning("No valid equipments found for template extraction")


2025-01-25 18:37:37,028  [INFO]: Extract: Templates - Equipments [Indicator Groups > Indicators]

2025-01-25 18:37:37,030  [DEBU]: SELECT DISTINCT "V_PAI_EQU_EXTERNAL_DATA".id, "V_PAI_EQU_EXTERNAL_DATA"."externalId", "V_PAI_EQU_EXTERNAL_DATA"."templateId", "V_PAI_EQU_EXTERNAL_DATA".valid 
FROM "V_PAI_EQU_EXTERNAL_DATA" 
WHERE "V_PAI_EQU_EXTERNAL_DATA".tenantid = :tenantid_1 AND "V_PAI_EQU_EXTERNAL_DATA".valid = :valid_1 ORDER BY "V_PAI_EQU_EXTERNAL_DATA"."externalId"
2025-01-25 18:37:37,032  [INFO]: [DB] SELECT V_PAI_EQU_EXTERNAL_DATA: 4
2025-01-25 18:37:37,033  [INFO]: Unique Templates linked to TOs: 3
2025-01-25 18:37:37,048  [INFO]: [GET] Templates based on Template ID
2025-01-25 18:37:37,469  [DEBU]: [GET] Template for Template ID 05D58DF9673046B487D4105DBE816F5B
2025-01-25 18:37:37,490  [DEBU]: [GET] Template for Template ID 9D72A836372246B3B535C5F1EB57AB31
2025-01-25 18:37:37,520  [DEBU]: [GET] Template for Template ID 98A93BA307024D0E883289396850EECC
2025-01-25 18:37:37,521  [I

In [45]:
# [EQU] Indicators & Indicator Groups Extraction
# ------------------------------------------------------------------------------ #

# Custom Imports
from modules.util.database import SQLAlchemyClient,EquIndicatorGroups, EquIndicators

if len(df_template_normal) > 0:
    file_ind_groups = rf'{EXTRACTION_DIR}/3_Template_EQU_Indicator_Groups.parquet'
    file_ind = rf'{EXTRACTION_DIR}/3_Template_EQU_Indicators.parquet'

    log.info(f'Extract: indicator groups & indicators for templates: {len(df_template_normal)}')

    ######### indicator groups
    cols = ['templateId', 'id', 'indicatorGroups']
    df_ind_groups = df_template_normal[cols].copy()
    df_ind_groups.dropna(subset=['indicatorGroups'], inplace=True)
    df_ind_groups.reset_index(drop=True, inplace=True)

    log.info(f"Records with indicator groups: {len(df_ind_groups)}")

    # Ensure indicatorGroups is a list of dictionaries
    df_ind_groups['indicatorGroups'] = df_ind_groups['indicatorGroups'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df_ind_groups_explode = explode_normalize(
        data=df_ind_groups,
        id_explode='indicatorGroups',
        id=['templateId', 'id']
    )

    df_ind_groups_explode = convert_dataframe(df_ind_groups_explode)
    log.info(f"Extract: Indicator Groups for templates: {len(df_ind_groups_explode)}")
    
    df_ind_groups_explode.to_parquet(file_ind_groups)
    log.info(f"{file_ind_groups} generated.")


    ######### indicators

    cols = ['templateId','id','indicatorGroups_id','indicatorGroups_internalId','indicatorGroups_indicators']
    df_indicators = df_ind_groups_explode[cols].copy()
    df_indicators.rename(columns={'indicatorGroups_indicators':'indicators'}, inplace=True)
    df_indicators.dropna(subset=['indicators'], inplace=True)
    df_indicators.reset_index(drop=True, inplace=True)

    log.info(f"Records with indicators: {len(df_indicators)}")

    df_indicators['indicators'] = df_indicators['indicators'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df_indicators_explode = explode_normalize(
        data=df_indicators,
        id_explode='indicators',
        id=['templateId', 'id', 'indicatorGroups_id', 'indicatorGroups_internalId']
    )
    df_indicators_explode = convert_dataframe(df_indicators_explode)
    log.info(f"EXTRACT: Indicators from Indicator Groups: {len(df_indicators_explode)}")
    
    df_indicators_explode.to_parquet(file_ind)
    log.info(f"{file_ind} generated.")

    if db.drop_reload:
        db.truncate(EquIndicatorGroups)  # truncate all existing data (remove if needed)
        db.truncate(EquIndicators)  # truncate all existing data (remove if needed)

    indicator_groups = SQLAlchemyClient.dataframe_to_object(df_ind_groups_explode, EquIndicatorGroups)
    if indicator_groups:
        db.insert_batches(data=indicator_groups)  # insert the new indicator groups

    indicators = SQLAlchemyClient.dataframe_to_object(df_indicators_explode, EquIndicators)
    if indicators:
        db.insert_batches(data=indicators)  # insert the new indicators
else:
    log.warning("No templates found for indicator extraction")

2025-01-25 18:37:37,549  [INFO]: Extract: indicator groups & indicators for templates: 3
2025-01-25 18:37:37,551  [INFO]: Records with indicator groups: 3
2025-01-25 18:37:37,561  [INFO]: Extract: Indicator Groups for templates: 5
2025-01-25 18:37:37,567  [INFO]: ../migration-data/indicators/extract/3_Template_EQU_Indicator_Groups.parquet generated.
2025-01-25 18:37:37,568  [INFO]: Records with indicators: 5
2025-01-25 18:37:37,579  [INFO]: EXTRACT: Indicators from Indicator Groups: 22
2025-01-25 18:37:37,591  [INFO]: ../migration-data/indicators/extract/3_Template_EQU_Indicators.parquet generated.
2025-01-25 18:37:37,592  [WARN]: [DB] TRUNCATE T_PAI_EQU_INDICATOR_GROUPS: 0
2025-01-25 18:37:37,593  [WARN]: [DB] TRUNCATE T_PAI_EQU_INDICATORS: 0
2025-01-25 18:37:37,594  [INFO]: [DB] INSERT T_PAI_EQU_INDICATOR_GROUPS: 5
2025-01-25 18:37:37,599  [INFO]: [DB] INSERT T_PAI_EQU_INDICATORS: 22


# 4 - FLOC Templates (Indicator Groups & Indicators)

To extract template information for Functional Locations, we follow these steps:

1. **Fetch Template Data:**
    - We fetch template data from the view `V_PAI_FlocExternalData` for Functional Locations that are valid for processing.

2. **Extract Templates:**
    - Using the fetched template data, we call the API to get detailed information about each template.
    - The extracted template details are normalized and stored in parquet files for further processing.

3. **Handle Errors:**
    - Any errors encountered during the API calls are logged and stored in CSV files for review.

4. **Stage Data in Tables:**
    - The extracted template details are then staged in the corresponding tables in the database.

5. **Extract Indicator Groups and Indicators:**
    - For each template, we extract the associated indicator groups and indicators.
    - The extracted data is normalized and stored in parquet files.
    - The data is then staged in the corresponding tables in the database.

The following table summarizes the data extraction and staging process:

| Data | Table Name |
| ---- | ---------- |
| Template Header for Functional Locations | `T_PAI_FLOC_TEMPLATE_HEADER` |
| Indicator Groups for Functional Locations | `T_PAI_FLOC_INDICATOR_GROUPS` |
| Indicators for Functional Locations | `T_PAI_FLOC_INDICATORS` |

The extracted data is crucial for the subsequent steps, where we will process the indicators linked to these templates.

In [46]:
# [FLOC] Templates: Extraction
# ------------------------------------------------------------------------------ #

# Standard Imports
import pandas as pd

# Custom Imports
from modules.util.database import SQLAlchemyClient, V_PAI_FlocExternalData, FlocTemplateHeader
from modules.util.helpers import convert_dataframe

file_tgt = rf'{EXTRACTION_DIR}/4_Template_FLOC.parquet'
file_err = rf'{REPORTS_DIR}/4_Template_FLOC_Errors.csv'

df_template_normal = pd.DataFrame()

Logger.blank_line(log)
log.info("Extract: Templates - Functional Locations [Indicator Groups > Indicators]")
Logger.blank_line(log)

# fetch template data from the view for external data - functional locations

results = db.select(
    model=V_PAI_FlocExternalData,
    fields=['id', 'externalId', 'templateId', 'valid'],
    distinct=True,
    where=[V_PAI_FlocExternalData.valid == "X"],
    orderby=['externalId']
)

if len(results) > 0:

    # extract templates for the identified records
    response, error = extract_templates(results, templateColumn='templateId')

    df_template_normal = pd.json_normalize(response, sep='_')
    df_template_normal = convert_dataframe(df_template_normal)
    df_template_normal.rename(columns={'id': 'flocId'}, inplace=True)
    df_template_normal.to_parquet(file_tgt, index=False)
    log.info(f"{file_tgt} generated")

    if error is not None and error != []:
        df_err = pd.json_normalize(error)
        df_err.to_csv(file_err, index=False)
        log.error(f"{file_err} generated")

    if db.drop_reload:
        db.truncate(FlocTemplateHeader)  # truncate all existing data (remove if needed)
        
    templates = SQLAlchemyClient.dataframe_to_object(df_template_normal, FlocTemplateHeader)
    if templates:
        db.insert_batches(data=templates)  # insert the new templates
else:
    log.warning(f"No valid functional locations found for template extraction")


2025-01-25 18:37:37,618  [INFO]: Extract: Templates - Functional Locations [Indicator Groups > Indicators]

2025-01-25 18:37:37,619  [DEBU]: SELECT DISTINCT "V_PAI_FLOC_EXTERNAL_DATA".id, "V_PAI_FLOC_EXTERNAL_DATA"."externalId", "V_PAI_FLOC_EXTERNAL_DATA"."templateId", "V_PAI_FLOC_EXTERNAL_DATA".valid 
FROM "V_PAI_FLOC_EXTERNAL_DATA" 
WHERE "V_PAI_FLOC_EXTERNAL_DATA".tenantid = :tenantid_1 AND "V_PAI_FLOC_EXTERNAL_DATA".valid = :valid_1 ORDER BY "V_PAI_FLOC_EXTERNAL_DATA"."externalId"
2025-01-25 18:37:37,620  [INFO]: [DB] SELECT V_PAI_FLOC_EXTERNAL_DATA: 1
2025-01-25 18:37:37,621  [INFO]: Unique Templates linked to TOs: 1
2025-01-25 18:37:37,636  [INFO]: [GET] Templates based on Template ID
2025-01-25 18:37:37,911  [DEBU]: [GET] Template for Template ID 9D72A836372246B3B535C5F1EB57AB31
2025-01-25 18:37:37,912  [INFO]: Templates fetched: 1
2025-01-25 18:37:37,922  [INFO]: ../migration-data/indicators/extract/4_Template_FLOC.parquet generated
2025-01-25 18:37:37,923  [WARN]: [DB] TRUNCA

In [47]:
# [FLOC] Indicators & Indicator Groups Extraction
# ------------------------------------------------------------------------------ #

# Custom Imports
from modules.util.database import SQLAlchemyClient, FlocIndicatorGroups, FlocIndicators

if len(df_template_normal) > 0:

    file_ind_groups_floc = rf'{EXTRACTION_DIR}/4_Template_FLOC_Indicator_Groups.parquet'
    file_ind_floc = rf'{EXTRACTION_DIR}/4_Template_FLOC_Indicators.parquet'

    log.info(f'Extract: indicator groups & indicators for templates: {len(df_template_normal)}')

    ######### indicator groups
    cols_floc = ['templateId', 'flocId', 'indicatorGroups']
    df_ind_groups_floc = df_template_normal[cols_floc].copy()
    df_ind_groups_floc.dropna(subset=['indicatorGroups'], inplace=True)
    df_ind_groups_floc.reset_index(drop=True, inplace=True)

    log.info(f"Records with indicatorGroups: {len(df_ind_groups_floc)}")

    # Ensure indicatorGroups is a list of dictionaries
    df_ind_groups_floc['indicatorGroups'] = df_ind_groups_floc['indicatorGroups'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df_ind_groups_explode_floc = explode_normalize(
        data=df_ind_groups_floc,
        id_explode='indicatorGroups',
        id=['templateId', 'flocId']
    )

    df_ind_groups_explode_floc = convert_dataframe(df_ind_groups_explode_floc)
    log.info(f"Extract: Indicator Groups for templates: {len(df_ind_groups_explode_floc)}")
    df_ind_groups_explode_floc.to_parquet(file_ind_groups_floc)
    log.info(f"{file_ind_groups_floc} generated.")


    # ######### indicators

    cols_floc = ['templateId', 'flocId', 'indicatorGroups_id', 'indicatorGroups_internalId', 'indicatorGroups_indicators']
    df_indicators_floc = df_ind_groups_explode_floc[cols_floc].copy()
    df_indicators_floc.rename(columns={'indicatorGroups_indicators': 'indicators'}, inplace=True)
    df_indicators_floc.dropna(subset=['indicators'], inplace=True)
    df_indicators_floc.reset_index(drop=True, inplace=True)

    log.info(f"Records with indicators: {len(df_indicators_floc)}")

    df_indicators_floc['indicators'] = df_indicators_floc['indicators'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df_indicators_explode_floc = explode_normalize(
        data=df_indicators_floc,
        id_explode='indicators',
        id=['templateId', 'flocId', 'indicatorGroups_id', 'indicatorGroups_internalId']
    )
    df_indicators_explode_floc = convert_dataframe(df_indicators_explode_floc)
    log.info(f"EXTRACT: Indicators from Indicator Groups: {len(df_indicators_explode_floc)}")
    df_indicators_explode_floc.to_parquet(file_ind_floc)
    log.info(f"{file_ind_floc} generated.")

    if db.drop_reload:
        db.truncate(FlocIndicatorGroups)  # truncate all existing data (remove if needed)
        db.truncate(FlocIndicators)  # truncate all existing data (remove if needed)

    indicator_groups_floc = SQLAlchemyClient.dataframe_to_object(df_ind_groups_explode_floc, FlocIndicatorGroups)
    if indicator_groups_floc:
        db.insert_batches(data=indicator_groups_floc)  # insert the new indicator groups

    indicators_floc = SQLAlchemyClient.dataframe_to_object(df_indicators_explode_floc, FlocIndicators)
    if indicators_floc:
        db.insert_batches(data=indicators_floc)  # insert the new indicators
else:
    print("WARN:\tNo templates found for indicator extraction")

2025-01-25 18:37:37,938  [INFO]: Extract: indicator groups & indicators for templates: 1
2025-01-25 18:37:37,940  [INFO]: Records with indicatorGroups: 1
2025-01-25 18:37:37,946  [INFO]: Extract: Indicator Groups for templates: 1
2025-01-25 18:37:37,951  [INFO]: ../migration-data/indicators/extract/4_Template_FLOC_Indicator_Groups.parquet generated.
2025-01-25 18:37:37,952  [INFO]: Records with indicators: 1
2025-01-25 18:37:37,959  [INFO]: EXTRACT: Indicators from Indicator Groups: 8
2025-01-25 18:37:37,967  [INFO]: ../migration-data/indicators/extract/4_Template_FLOC_Indicators.parquet generated.
2025-01-25 18:37:37,968  [WARN]: [DB] TRUNCATE T_PAI_FLOC_INDICATOR_GROUPS: 0
2025-01-25 18:37:37,969  [WARN]: [DB] TRUNCATE T_PAI_FLOC_INDICATORS: 0
2025-01-25 18:37:37,970  [INFO]: [DB] INSERT T_PAI_FLOC_INDICATOR_GROUPS: 1
2025-01-25 18:37:37,972  [INFO]: [DB] INSERT T_PAI_FLOC_INDICATORS: 8
